LSST Catalog Outputs Demo
=======

General Goals
------------

- Load the output sources table from process `processCcdDecam.py`
- Set up the butler, use it to access output products
- Display the output image, and possibly overlay sources?

Initial setup for this notebook
-----------------------------

This loads the shared LSST environment, with a recent-ish tag, and a git repo with `obs_decam`. The call to load anaconda separately is due to some bug in loadLSST.sh where an old anaconda version gets loaded. 

    source /lsst/stack/loadLSST.sh
    setup anaconda -t b1628
    setup -r. obs_decam -t b1628
    
We will also load some packages and specify a base directory that we will load files from:

In [22]:
import math
import lsst.afw.table as afwTable
import lsst.daf.persistence as dafPersist

repo_dir = "/raid/ctslater/decam_NEO_repo"

Reading Tables
-------------
First I'll do the basic (perhaps "naiëve" or "unsophisticated") of reading files, by directly specifying a filename.

In [50]:
cat = afwTable.SourceCatalog.readFits(repo_dir + "/0197828/src/src-0197828_09.fits")
print cat

<lsst.afw.table.tableLib.SourceCatalog; proxy of <Swig Object of type 'lsst::afw::table::SortedCatalogT< lsst::afw::table::SourceRecord > *' at 0x7ff8fed91420> >


**Complaint:** The print function output here is very useless. It would be much more useful to see a snippet of the table contents.

But there are ways to see the contents. Most usefully, `cat.schema`

In [51]:
cat.schema

Schema(
    (Field['L'](name="id", doc="unique ID"), Key<L>(offset=0, nElements=1)),
    (Field['Angle'](name="coord_ra", doc="position in ra/dec"), Key<Angle>(offset=8, nElements=1)),
    (Field['Angle'](name="coord_dec", doc="position in ra/dec"), Key<Angle>(offset=16, nElements=1)),
    (Field['L'](name="parent", doc="unique ID of parent source"), Key<L>(offset=24, nElements=1)),
    (Field['Flag'](name="calib_detected", doc="Source was detected as an icSrc"), Key['Flag'](offset=32, bit=0)),
    (Field['Flag'](name="calib_psfCandidate", doc="Flag set if the source was a candidate for PSF determination, as determined by the"), Key['Flag'](offset=32, bit=1)),
    (Field['Flag'](name="calib_psfUsed", doc="Flag set if the source was actually used for PSF determination, as determined by the"), Key['Flag'](offset=32, bit=2)),
    (Field['Flag'](name="flags_negative", doc="set if source was detected as significantly negative"), Key['Flag'](offset=32, bit=3)),
    (Field['D'](name="base_Gau

This may look like a bit of a mess, but that's fine because it's at least telling me what is in the file.

We can access individual sources:

In [52]:
s = cat[0]
print s

<lsst.afw.table.tableLib.SourceRecord; proxy of <Swig Object of type 'boost::shared_ptr< lsst::afw::table::SourceRecord > *' at 0x7ff8fed914b0> >


**Complaint:** Also not very pretty output from print

In [25]:
print "ID: ", s['id']
print "RA: ", math.degrees(s.get('coord_ra'))
print "Dec: ", math.degrees(s.get('coord_dec'))
print "ApFlux: ", s.getApFlux()


ID:  217514186299671035
RA:  196.296482023
Dec:  -17.8391624357
ApFlux:  -13152.8325061


So there we see three ways, given a single source `s`, we can either get its fields via brackets, via `.get()`, or via any getters that may be defined.

These methods will also work on the table itself, giving us the entire column:

In [53]:
cat.getApFlux()

array([-13152.83250606,  -5499.91841248,  -5922.28439512, ...,
          336.13636792,    496.06406267,  -6457.87151962])

Perfect! Data we can use.

We can see many of the getters by calling `help()`. However, **Complaint:** `help(cat)` on the whole table does not show the getters that are accessible but only shown in `help(s)` on an individual entry.

In [45]:
help(s)

Help on SourceRecord in module lsst.afw.table.tableLib object:

class SourceRecord(SimpleRecord)
 |  Proxy of C++ lsst::afw::table::SourceRecord class
 |  
 |  Method resolution order:
 |      SourceRecord
 |      SimpleRecord
 |      BaseRecord
 |      __builtin__.object
 |  
 |  Methods defined here:
 |  
 |  __del__ lambda self
 |  
 |  __getattr__ lambda self, name
 |  
 |  __init__(self, *args, **kwargs)
 |  
 |  __repr__ = _swig_repr(self)
 |  
 |  __setattr__ lambda self, name, value
 |  
 |  getApFlux(self)
 |      getApFlux(SourceRecord self) -> lsst::afw::table::FluxSlotDefinition::MeasValue
 |  
 |  getApFluxErr(self)
 |      getApFluxErr(SourceRecord self) -> lsst::afw::table::FluxSlotDefinition::ErrValue
 |  
 |  getApFluxFlag(self)
 |      getApFluxFlag(SourceRecord self) -> bool
 |  
 |  getCalibFlux(self)
 |      getCalibFlux(SourceRecord self) -> lsst::afw::table::FluxSlotDefinition::MeasValue
 |  
 |  getCalibFluxErr(self)
 |      getCalibFluxErr(SourceRecord self) ->

The Butler
----------

Ok, this previous example was slightly cheating (or at least not being as fancy as we could be) by specifying filenames directly. Let's instead access them through the butler.

In [28]:
b = dafPersist.Butler(repo_dir)

In [34]:
print b
help(b.get)

Help on method get in module lsst.daf.persistence.butler:

get(self, datasetType, dataId={}, immediate=False, **rest) method of lsst.daf.persistence.butler.Butler instance
    Retrieves a dataset given an input collection data id.
    
    @param datasetType (str)   the type of dataset to retrieve.
    @param dataId (dict)       the data id.
    @param immediate (bool)    don't use a proxy for delayed loading.
    @param **rest              keyword arguments for the data id.
    @returns an object retrieved from the dataset (or a proxy for one).



datasetType is one of the filenames that exist for each visit, in this case we want "src", but it could also be "instcal" or "calexp" for images. Let's try that

In [41]:
cat_butler = b.get("src", visit=197828, ccdnum=9)
print cat_butler

<lsst.afw.table.tableLib.SourceCatalog; proxy of <Swig Object of type 'lsst::afw::table::SortedCatalogT< lsst::afw::table::SourceRecord > *' at 0x7ff8fed91390> >


In [42]:
cat_butler.getApFlux()

array([-13152.83250606,  -5499.91841248,  -5922.28439512, ...,
          336.13636792,    496.06406267,  -6457.87151962])

Same output as before, just as we hoped for.

Now, what if we want to know more about the data available in the repository. For instance, all the visits:

In [49]:
print b.queryMetadata("src", "visit")

[197383, 197387, 197391, 197395, 197824, 197828, 197832, 197836, 198405, 198409, 198413, 198417, 198762, 198766, 198770, 198774]


Or the ccds available

In [48]:
print b.queryMetadata("src", "ccdnum", visit=197383)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 62]


Not all of these actually exist; while visit 197383 does have instcal, wtmap, and dqmask images, it doesn't actually have "src" sources files. So I'm not clear on the exact semantics of what the query is supposed to be returning here.

What happens if I ask for something that doesn't exist?

In [57]:
nonexistant = b.get("src", visit=197383, ccdnum=10)

Ok?

In [60]:
print nonexistant            

RuntimeError: No such FITS catalog file: /raid/ctslater/decam_NEO_repo/0197383/src/src-0197383_10.fits

Neat, clearly the butler was doing some sort of delayed loading thing? It's not clear to me if the butler is supposed to fail on `get()` for nonexistent objects or not, but as is, it doesn't mind.